<a href="https://colab.research.google.com/github/walkymatt/BCMD/blob/master/lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP0161 Auditory Computing Week 4: Computational Auditory Scene Analysis

In this week's tutorial we'll be looking at some tools for finding relationships within an audio signal. These relationships can help identify elements in the signal that are part of the same **object** or **stream**.

Human auditory perception tends to be *really* good at this. We are able to isolate, group and attend to complex sounds in the midst of many distractions using a variety of cues. You will have heard some examples in this week's lecture, and we'll do some more in the tutorial.

This task — often metonymised as the **cocktail party problem** — is computationally challenging and a variety of approaches have been attempted. **Computational Auditory Scene Analysis** (CASA) is a subset of these where the mechanics are modelled on ways in which human perceptions work.

Once again we'll be using [Google's Colab computing environment](https://colab.research.google.com/#) and with [Python](https://docs.python.org/3/tutorial/index.html). As usual code is provided, so you should be able to follow along even if you are unfamiliar with Python. Refer to last week's notebook for a very brief intro to Colab.

# Setting Up

In addition to the general purpose libraries we used last time ([NumPy](https://numpy.org/doc/stable/), [SciPy](https://docs.scipy.org/doc/scipy/index.html), [MatPlotLib](https://matplotlib.org/stable/users/index), [IPython.display](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html)), we'll make use of a more specialised audio package, [pyfar](https://pyfar-gallery.readthedocs.io/en/latest/).

This does not come pre-installed on Colab, so we'll need to install it explicitly.

In [ ]:
%pip install pyfar

As usual, we'll need to import the packages we want to use so the Python knows it's using them.

In [ ]:
import numpy as np
import scipy.signal as ssg
import scipy.fft as fft
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Image, Audio

import pyfar as pf

# we'll be wanting some random numbers later
# we create a generator with a fixed seed for consistent behaviour
# if you want different results, change the seed
SEED = 9908
rng = np.random.default_rng(SEED);

Finally, fetch an audio file we're going to use later on.

In [ ]:
!curl -O https://comp0161.github.io/assets/data/streams-mono.wav

# Convolution & Correlation

An important operation in digital signal processing is **convolution**, which is a way of combining two signals together.

In the continuous domain, convolution looks like this:

$$
(f \otimes g)(t) = \int_{-\infty}^{\infty} f(s) g(t-s) \, \mathrm{d}s
$$

The functions $f$ and $g$ represent two signals and the convolution operation $\otimes$ produces a new signal $(f \otimes g)$. The important things to note are:

* the value of the convolved function at any time $t$ integrates over the entirety of both $f$ and $g$, at a fixed offset $t$ from one another
* so, varying $t$ equates to changing the position of — or *sliding* — one signal relative to the other
* in the integral with respect to $s$, $f$ varies with $s$ while $g$ varies with $-s$ — which means one of the signals is *reversed in time* relative to the other

A very closely related operation is known as the **cross-correlation** function (CCF), and it looks like this:

$$
(f \star g)(t) = \int_{-\infty}^{\infty} f(s) g(t+s) \, \mathrm{d}s
$$

It's essentially the same operation without the time reversal.

Digital signals are discrete and finite, so instead of an integral over infinity we use a finite sum:

$$
(f \otimes g)_t = \sum_{s=0}^S f_s g_{t-s}
$$

<details><summary>Note</summary>
I'm glossing over the range and indexing a bit here, but essentially this is computed over the shared extent of the two signals.
</details>

Okay, but why do we care about any of this? It probably all seems a bit abstract, but it turns out that there are a lot of situations where we're interested in something that combines information from different time points in a signal.

For example, compare the above with last week's difference equation for a **finite impulse response filter**:

$$
y_t = \sum_{i=0}^P b_i x_{t-i}
$$

**It's the same thing!** In fact an FIR filter is an example of convolution in action.

Another example is **reverberation**: the sound we hear at any given time in a reverberant space includes echoes of earlier sounds bouncing back from the different surfaces in the space. If we know how the space responds to a single instantaneous sound impulse, then we can compute what any signal will sound like in the space by convolving it with the impulse response. We'll  come back to this idea of **convolution reverb** in week 6.

Cross-correlation is very similar to convolution — it slides one signal along the other and computes the inner product between them at each position. But because there is no time reversal it can be considered as computing a **similarity metric** between the two signals each time. When the two signals are very similar, high points in each multiply together and the result is maximised; when they are different the lows multiply the highs and the output is reduced. Consequently, it can be used for **coincidence detection**: peaks in the CCF correspond to times when the two signals are optimally aligned.

A special case of cross-correlation is the **autocorrelation function** (ACF), which is just the cross-correlation of a signal with itself. Again, this is computing a similarity function, but this time with the signal's own past and/or future. So it is useful for detecting patterns of recurrence or **periodicity**.

We'll look at some examples below.

## Aside: Computational Complexity

Convolving two signals requires computing the weighted sum between them at every possible offset, which can become computationally expensive for long signals. But there's an important fact about convolution that can come to the rescue, at least a bit, at least some of the time.

This fact is so important it is literally called the **Convolution Theorem**:

>Convolution in the time domain is equivalent to multiplication in the frequency domain, and vice versa.

This may be expressed as:

$$
\mathcal{F}(f \otimes g) = \mathcal{F}(f) \mathcal{F}(g)
$$

or equivalently:

$$
f \otimes g = \mathcal{F}^{-1}(\mathcal{F}(f)\mathcal{F}(g))
$$

where $\mathcal{F}$ denotes the Fourier transform.

What this means computationally is that you can compute the convolution of two signals by performing an FFT on each, multiplying the results elementwise, and then performing an inverse FFT on that product.

None of these operations is free, but the FFTs are $O(n \log n)$ whereas the convolution is $O(n^2)$, so for large signal sizes it can be faster to take the detour through frequency space.

## Examples


In [ ]:
# some simple example signals
tt = np.arange(1024) * 8 * np.pi / 1024

f0 = pf.Signal(np.sin(tt), 44100)
h2 = pf.Signal(np.sin(tt*2), 44100)
h3 = pf.Signal(np.sin(tt*3), 44100)
h4 = pf.Signal(np.sin(tt*4), 44100)
h5 = pf.Signal(np.sin(tt*5), 44100)

sawish = f0 + (h2/2) + (h3/3) + (h4/4) + (h5/5)
noise = pf.signals.noise(1024)

Consider a very rough approximation of a sawtooth wave, built from a fundamental and four higher harmonics:

In [ ]:
pf.plot.time_freq(sawish);

Let's compute the ACF for this signal. (Note that we're doing this with `convolve`, which implicitly time reverses one of the signals. But we want to correlate, so we need to reverse it back. In this case the waveform is odd and the signal contains a whole number of cycles, so negating happens to have the same effect as reversing time. This would not be true in general.)

In [ ]:
self_saw = pf.dsp.convolve(sawish, -1 * sawish, mode='cyclic')
pf.plot.time_freq(self_saw);

The peaks in the ACF clearly line up with the period of the waveform, and we can read the fundamental off from the highest peak in the spectrum.

OK, but this isn't super exciting because both of those things were already true of the original signal, without recourse to the ACF. Let's try a potentially trickier variation:

In [ ]:
noisy_saw = sawish + 3 * noise
pf.plot.time_freq(noisy_saw);

Here we've added a load of white noise. The are some hints of the underlying periodicity in the time domain data, and the fundamental is just visible as the highest peak in the spectrum, but it's pretty murky.

However, the ACF pulls it out a lot more clearly.

(Note that the noise is not rotationally symmetrical, so we can't use the negation trick this time.)

In [ ]:
noisy_saw_acf = pf.dsp.convolve(noisy_saw, pf.Signal(noisy_saw.time[0,::-1], 44100), mode='cyclic')
pf.plot.time_freq(noisy_saw_acf);

We can use cross-correlation to look for alignment between two signals. For instance, here is the CCF for the fourth harmonic within the saw wave:

In [ ]:
saw_h4_ccf = pf.dsp.convolve(sawish, -1 * h4, mode='cyclic')
pf.plot.time_freq(saw_h4_ccf);

The harmonic occurs throughout, of course, but its CCF value varies with the lag. It's at its maximum when the harmonic is in phase, and at its minimum when out of phase. We can see the first peak is at zero, which makes sense because the harmonic is present in the saw with no phase shift.

We can construct a marginally more interesting example piecewise:

In [ ]:
env = (1 - np.cos(tt[:256]))/2
blip = env * np.sin(tt[:256])
plt.plot(blip);

Our signal will contain this shape a few times.

In [ ]:
blips = np.zeros(1024)
blips[100:356] += blip
blips[500:756] += blip
blips[650:906] += blip

sig_blips = pf.Signal(blips, 44100)
pf.plot.time_freq(sig_blips);

We're going to search for this more than once, so let's go ahead and construct a time-flipped signal upfront. Also, we're going to pad it to the same length as the target to make sure the alignment is correct.

In [ ]:
pad_blip = np.concatenate((blip, np.zeros(768)))
sig_blip = pf.Signal(pad_blip[::-1], 44100)
pf.plot.time_freq(sig_blip);

Ok, let's use CCF to search for matches.

In [ ]:
blips_ccf = pf.dsp.convolve(sig_blips, sig_blip, mode='cyclic')
pf.plot.time_freq(blips_ccf);

The peaks in the CFF correspond to the times where the thing we're searching for was found.

Let's try the same thing with some noise:

In [ ]:
noisy_blips = sig_blips + noise
pf.plot.time_freq(noisy_blips);

That's pretty noisy. You can just about make out the peaks, but I don't think you'd locate them with great confidence. The CCF does a pretty good job, however:

In [ ]:
noisy_blips_ccf = pf.dsp.convolve(noisy_blips, sig_blip, mode='cyclic')
pf.plot.time_freq(noisy_blips_ccf);

There are arguably some spurious peaks, especially at the end, though they're a lot lower and could be thresholded out. But the three true peaks are correctly localised.


# Modulation

Information can be carried in an audio signal in a number of ways. In general, encoding information requires **change**. Sound is vibration, so it's never exactly static, but if its essential properties of frequency and amplitude remain constant then it doesn't convey very much. So typically, information is captured by variations — **modulation** — of these properties.

# Amplitude Modulation

In **amplitude modulation** (AM), information is carried by changes in the signal amplitude. An underlying waveform of higher frequency acts as the **carrier**, and its volume (or brightness or power or other measure of intensity) is varied to encode the message. (The carrier frequency does not need to be constant, though it will be for our purposes below, but it should be fast relative to the rate of variation in the message for similar reasons to the sampling theorem in the digital domain.)

AM is widely used in synthetic signals (AM radio is a famous example), but is also a common carrier of information in natural sounds such as speech — one of the ways in which the mouth shapes sound is by varying its volume.

For digital signals, we can easily encode a message (usually scaled into [0, 1] for simplicity) just by multiplying the carrier with it.

In [ ]:
RATE = 44100
INTERVAL = 1/RATE
N = 4410
CARRIER_FREQ = 1000

tt = np.arange(N) * INTERVAL
carrier_phase = CARRIER_FREQ * 2 * np.pi * tt
carrier = np.sin(carrier_phase)

message = (np.sin(np.cumsum(tt)/10) + 1) / 2
am = carrier * message

Here's the carrier, a plain sine wave:

In [ ]:
plt.plot(carrier);

Our message is also pretty simple, a lower frequency sine sweep:

In [ ]:
plt.plot(message);

And the AM signal is just the elementwise product of the two:

In [ ]:
plt.plot(am);

The message is clearly visible in the combined signal, but if we have a look at the spectra of the three signals, we can see that the spectrum of the modulated signal is massively dominated by carrier frequency.

(Recalling the convolution theorem, can you see why that would be?)

In [ ]:
pf.plot.time_freq(pf.Signal(carrier, RATE));

In [ ]:
pf.plot.time_freq(pf.Signal(message, RATE));

In [ ]:
pf.plot.time_freq(pf.Signal(am, RATE));

We can recover the message fairly easily, we just need to get rid of that carrier.

We first **rectify** the signal to avoid the positive and negative swings cancelling each other out. (Hair cells in the ear perform a similar function.)

Two common approaches to this are **half-wave rectification**, in which negative-going swings are discarded (set to zero), and **full-wave rectification**, in which negative swings are flipped to positive. Half-wave is more physiological and slightly easier to implement in analogue circuits, while full-wave is more efficient (it preserves more of the signal power) and may offer marginally better fidelity. Both are easy to implement computationally.

In [ ]:
half_rect = np.where(am > 0, am, 0)
full_rect = np.abs(am)

In [ ]:
plt.plot(half_rect);

In [ ]:
plt.plot(full_rect);

The message spectrum is visible in the lower frequency range of the rectified spectra, while the carrier is in the upper range.

(Notice that the first carrier peak in the full-wave spectrum is twice the frequency of the half-wave version. Flipping the negative swings positive effectively doubles the frequency.)

In [ ]:
pf.plot.time_freq(pf.Signal(half_rect, RATE));

In [ ]:
pf.plot.time_freq(pf.Signal(full_rect, RATE));

At this point, all we need to do to recover the message is get rid of all those higher frequencies from the carrier. This calls for a low pass filter.

In [ ]:
half_filt = pf.dsp.filter.butterworth(pf.Signal(half_rect, RATE), 10, 500)
full_filt = pf.dsp.filter.butterworth(pf.Signal(full_rect, RATE), 10, 500)

In [ ]:
pf.plot.time_freq(half_filt);

In [ ]:
pf.plot.time_freq(full_filt);

Both versions recover the signal pretty nicely in this case. There's some ripple at the beginning, which is a boundary effect of the filtering. In both cases we get smaller amplitude than the original signal due to losses along the way, and this is twice as bad for the half-rectification, which throws away more signal.

# Frequency Modulation

As the name suggests, in **frequency modulation** (FM), the message is carried by changes in frequency rather than amplitude. This is a little more difficult to implement, but can be more robust. For example, low values in the message do not manifest as a lower signal strength, as they would with AM.

Like AM, FM is commonly used for synthetic signals (radio is again an obvious example) but also occurs widely in natural sounds such as speech. For example, inflection is often an important signifier in spoken language — the change is pitch carries meaning, rather than whatever baseline pitch it started out at.

Once again, we'll illustrate with a simple example, using the same message and carrier frequency as for AM.

In [ ]:
# time base, rate and message are all the same as before

# carrier frequency is also the same, but this time we'll
# modulate the carrier_phase before converting to a sine
# rather than modulating the sine directly

# scaling factor for how signal level modulates phase
sensitivity = 0.1
modulated_phase = carrier_phase + sensitivity * np.cumsum(message)

fm = np.sin(modulated_phase)

In [ ]:
plt.plot(fm);

The spectrum of the FM signal is still strongly affected by the carrier, but it's not so dominant because that frequency varies with the message rather than being constant.

In [ ]:
pf.plot.time_freq(pf.Signal(fm, RATE));

To decode the message we need to estimate the frequency of the varying signal, which is a little less obvious than estimating the amplitude.

One way to do it is to look at the **gradient** of the waves. The gradient of a sinusoidal oscillation is constantly changing, but its range of values depends on the frequency: higher frequency waves have shorter wavelengths and are therefore (for a given amplitude) steeper. We can estimate the gradient at each sample numerically by comparison with its neighbours:

In [ ]:
plt.plot(np.gradient(fm));

In [ ]:
pf.plot.time_freq(pf.Signal(np.gradient(fm), RATE));

The message is effectively *AM encoded* in the FM gradient, and we can recover it by rectification and filtering just as before:

In [ ]:
full_filt_grad = pf.dsp.filter.butterworth(pf.Signal(np.abs(np.gradient(fm)), RATE), 10, 500)

In [ ]:
pf.plot.time_freq(full_filt_grad);

The gradient approach is neat, but probably the most common method for FM decoding these kind of sampled audio signals is to count **zero crossing** points. The carrier oscillates over a symmetric range around zero, so crossings occur every half cycle and their timing gives an estimate of the period — and hence the frequency — over that half wave.

In [ ]:
# find zero crossings
positive = fm > 0
crossings = np.flatnonzero(np.bitwise_xor(positive[1:], positive[:-1]))
crossing_times = crossings * INTERVAL

# carrier is symmetrical, so each crossing should approximate 1/2 a cycle
crossing_half_waves = np.diff(np.concatenate(([0], crossing_times)))
crossing_freqs = 1/(2 * crossing_half_waves)

# modulation is relative to the carrier baseline
mod_freqs = crossing_freqs - CARRIER_FREQ

# plot at the crossing times
plt.plot(crossing_times, mod_freqs);

One thing to notice is that our time resolution is limited by our sampling frequency, so the raw FM message we recover here is *stepped*. But the quantisation constitutes a kind of **pulse width modulation**, which we can roughly decode with (again) a low pass filter.

In [ ]:
# first expand the runs so that we have an evenly sampled signal
reps = np.diff(crossings)
reps = np.concatenate([reps, [len(message) - np.sum(reps)]])
expanded_freqs = np.repeat(mod_freqs, reps)

pf.plot.time_freq(pf.Signal(expanded_freqs, RATE));

In [ ]:
# smooth out the pulses with an LPF
filt_zeros = pf.dsp.filter.butterworth(pf.Signal(expanded_freqs, RATE), 10, 500)

# plot the recovered message
pf.plot.time_freq(filt_zeros);

As with the AM decoding, there are some artefacts in the recovered message. Notably, the amplitude is completely garbled (it looks like there's a missing factor of 1000, which may be from the carrier frequency but I can't immediately think why that should be), and there's also some ringing from the filter at the start.

Even so, the message is reconstructed reasonably well, and we could almost certainly do better with only a little tweaking. (Feel free to take this as an exercise for the reader!)

# Computational Auditory Scene Analysis


Until now we have only considered very basic synthetic signals where the structure is known *a priori*. But in practice audio tends to be pretty messy, and we usually don't know exactly what is in it. Which is where **CASA** comes in (at least as one possible route).

How might we approach an actual audio signal in a CASA framework, bearing in mind the tools discussed above?

Let's load some audio to play with.

 <details>
 <summary>Notes</summary>
 <ul><li><small>This WAV file was generated with <a href="https://tidalcycles.org/">Tidal Cycles</a> using code run during the tutorial. The actual script will be linked from the <a href="https://comp0161.github.io">COMP0161 GitHub</a>.</small></li>
 <li><small>Saving to WAV is not directly supported in Tidal, so this was actually recorded with <a href="https://rogueamoeba.com/audiohijack/">Audio Hijack</a>, but there's probably a better way.</small></li>
 </ul>
 </details>


In [ ]:
mono = pf.io.read_audio('streams-mono.wav')

In [ ]:
display(Audio(mono.time, rate=48000))

This signal, like essentially all audio with any interesting content, contains a mixture of frequencies that varies over time. We know that information might be contained in the variations of frequency and amplitude, and that these variations unfold over the course of the signal. So we probably want to consider some kind of **time-frequency** representation.

One very common such representation is a **spectrogram**, which we saw last week. It essentially chops the signal into chunks and calculates the Fourier spectrum for each chunk. These spectra are then represented as columns in a plot where time advances horizontally:

In [ ]:
pf.plot.spectrogram(mono);

This representation is fine in many contexts — *great* actually — but it doesn't take into account the non-uniform frequency response of human hearing. In the CASA context we usually prefer a time-frequency decomposition that stratifies frequencies in a psychophysical way — that is, breaking them down into bands that seem perceptually commensurate to human listeners.

What this boils down to is applying a **bank** of bandpass filters to the signal, each one picking out some subset of frequencies that we think constitute a relevant signal component. The output is a set of parallel signals, each containing a different fraction of the original. Depending on the filters, it may or may not be possible to reconstruct the original signal from the filterbank outputs.

Many different filterbanks have been used for this, varying in the shape of the filters and spacing of their frequencies. Broadly, they each attempt to capture the **critical band** structure of human hearing in their own way. One popular one is the **mel scale** filterbank, which we will touch on next week. For the moment let's focus on another, the **gammatone** filterbank.

A [gammatone filter](https://en.wikipedia.org/wiki/Gammatone_filter) very approximately models the impulse response of hair cells in the cochlea. In the time domain it combines a simple sine wave (tone) with a [gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) envelope, resulting in a sort of wave packet. In the frequency domain this generates an asymmetric bandpass filter, as seen in the plot a couple of cells down.

The standard gammatone filter bank consists of a stack of gammatone filters with centre frequencies spaced to be perceptually equal.

In [ ]:
GFB = pf.dsp.filter.GammatoneBands(frequency_range=[20, 20000], sampling_rate=48000)

In [ ]:
# apply the filter bank to an impulse to get a picture of the responses
impulse_filter_bank, _ = GFB.process(pf.signals.impulse(4096, sampling_rate=48000))
ax = pf.plot.freq(impulse_filter_bank)
ax.set_ylim(-60, 5);

Applying this filterbank to our audio, we generate a bunch of parallel constituent signals in different perceptual bands. Each of them is a (band-limited) audio signal in its own right, which we can listen to (if we have the patience).

In [ ]:
# the gammatone filters produce complex outputs, don't worry about this for now
real, imag = GFB.process(mono)

In [ ]:
# each filter pulls out a subset of the frequency content of the signal
# we can listen to the bands individually
for ii in range(40):
  display(Audio(real[ii].time, rate=48000))

In the same way that the spectrogram captures the frequency variations in the signal over time on a flat scale, the gammatone filterbank outputs can be considered as decomposing the signal on a perceptually relevant scale — the **cochleagram**. We can present this similarly by aggregating over short time windows.

(NB: this is pretty computationally intensive and, unlike a lot of the above, has no optimised backend implementation. So it may take a long time to run — on the order a minute and a half in my tests.)

In [ ]:
# kludge up a cochleagram from the gammatone signals
# for the moment we'll divide into non-overlapping 20ms windows
# and just take the sum of squares for each
# (this is a crude proxy for the envelope, with summing over the window acting as a filter)
WINDOW = 48000 // 50
COLS = real[0].time.shape[1] // WINDOW
ROWS = real.cshape[0]
cochgram = np.zeros((ROWS, COLS))

for row in range(ROWS):
  for col in range(COLS):
    t1 = col * WINDOW
    t2 = (col + 1) * WINDOW
    chunk = real[row].time[0,t1:t2]
    cochgram[row, col] = np.sum(chunk**2)

In [ ]:
# range is very skewed, so we apply a log transform to compress it
log_cochgram = np.log(cochgram + .1)

# scale to pixel range and type to create an image
coch_img = ((log_cochgram - np.min(log_cochgram))/(np.max(log_cochgram) - np.min(log_cochgram)) * 255).astype(int)

In [ ]:
plt.imshow(coch_img, origin='lower', aspect='auto', cmap='nipy_spectral');

In [ ]:
tt = np.arange(COLS) / 50
plt.plot(tt, log_cochgram[20,:])

In [ ]:
# kludge up a correlogram
corrgram = np.zeros((ROWS, COLS))
for row in range(ROWS):
  acf = pf.dsp.convolve(pf.Signal(log_cochgram[row,:], 50), pf.Signal(log_cochgram[row,::-1], 50), mode='cyclic')
  corrgram[row,:] = (acf.time[0,:] - np.min(acf.time[0,:]))/(np.max(acf.time[0,:]) - np.min(acf.time[0,:]))


In [ ]:
plt.imshow(corrgram, origin='lower', aspect='auto');

In [ ]:
for row in range(ROWS):
  plt.plot(tt, corrgram[row,:] + row)